In [1]:
print("ok")

ok


In [36]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader

In [44]:
PINECONE_API_KEY="02a244d9-c183-4a70-8eac-6b67bc1f139f"


In [14]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [15]:
extracted_data=load_pdf("data/")

In [17]:
#extracted_data

In [18]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)
print("No. of chunks:", len(text_chunks))

No. of chunks: 7020


In [21]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [22]:
embeddings = download_hugging_face_embeddings()

In [23]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
#query_result

In [47]:
PINECONE_API_KEY

'02a244d9-c183-4a70-8eac-6b67bc1f139f'

In [98]:
#Initializing the Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chat-bot")

#Creating Embeddings for Each of The Text Chunks & storing
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

KeyboardInterrupt: 

In [113]:
index

In [124]:
vectorstore

In [114]:
len(docsearch)

406

In [60]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [102]:

# If we already have an index, we can load it like this
query = "What are Allergies"
query=embeddings.embed_query(query)
#print(query)
docs = index.query(namespace="real",
    vector=query,
    top_k=3,
    include_values=True
)

print("Result:", docs)


Result: {'matches': [{'id': '1372',
              'score': 0.682538629,
              'values': [0.0354718827,
                         -0.0110478215,
                         0.0751323774,
                         0.00805775635,
                         0.113756701,
                         0.0377156287,
                         0.113712087,
                         0.0840440467,
                         -0.0305279065,
                         0.0913646743,
                         0.0113536743,
                         -0.0614714548,
                         0.0239176191,
                         0.0503650978,
                         -0.0131362583,
                         0.0664992854,
                         -0.0492044911,
                         -0.0601874366,
                         -0.0266646389,
                         -0.048428636,
                         0.0071746828,
                         0.0825395435,
                         -0.0148052862,
                        

In [81]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [82]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [95]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [115]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

AttributeError: 'list' object has no attribute 'as_retriever'